In [1]:
import numpy as np
import pandas as pd

# Step 1: Create the Decision Matrix
data = {
    'Model': ['BART', 'PEGASUS', 'T5', 'GPT-3', 'LED'],
    'ROUGE-1': [44.5, 46.2, 42.8, 41.3, 43.9],  # Benefit
    'ROUGE-2': [21.7, 23.1, 20.4, 19.8, 20.9],  # Benefit
    'ROUGE-L': [42.0, 43.8, 40.1, 39.5, 41.3],  # Benefit
    'Inference Time': [0.7, 0.6, 0.9, 1.2, 0.8],  # Cost
    'Model Size': [406, 568, 850, 1500, 650],     # Cost
    'FLOPs': [150, 180, 200, 400, 250],           # Cost
    'Memory Usage': [4.5, 5.0, 6.0, 8.0, 5.5]     # Cost
}
df = pd.DataFrame(data)

In [2]:
# Step 2: Normalize the Decision Matrix
criteria = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'Inference Time', 'Model Size', 'FLOPs', 'Memory Usage']
benefit_criteria = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L']
cost_criteria = ['Inference Time', 'Model Size', 'FLOPs', 'Memory Usage']


# Normalization
norm_df = df.copy()
for col in criteria:
    if col in benefit_criteria:
        norm_df[col] = df[col] / np.sqrt((df[col] ** 2).sum())
    else:
        norm_df[col] = df[col].min() / df[col]

In [3]:
# Step 3: Apply Weights
weights = [0.20, 0.20, 0.15, 0.15, 0.10, 0.10, 0.10]
norm_df[criteria] = norm_df[criteria] * weights

# Step 4: Determine Ideal and Negative-Ideal Solutions
ideal_solution = norm_df[criteria].max()
negative_ideal_solution = norm_df[criteria].min()

# Step 5: Calculate Separation Measures
norm_df['S+'] = np.sqrt(((norm_df[criteria] - ideal_solution) ** 2).sum(axis=1))
norm_df['S-'] = np.sqrt(((norm_df[criteria] - negative_ideal_solution) ** 2).sum(axis=1))

# Step 6: Calculate Closeness Coefficient
norm_df['Closeness Coefficient'] = norm_df['S-'] / (norm_df['S+'] + norm_df['S-'])

In [4]:
# Step 7: Rank the Models
result = norm_df[['Model', 'Closeness Coefficient']].sort_values(by='Closeness Coefficient', ascending=False)

print(result)

     Model  Closeness Coefficient
0     BART               0.839756
1  PEGASUS               0.753906
4      LED               0.471522
2       T5               0.393957
3    GPT-3               0.000000


In [7]:
rank_report = result.copy()
rank_report['Rank'] = rank_report['Closeness Coefficient'].rank(ascending=False).astype(int)
rank_report = rank_report.sort_values(by='Rank')

print("TOPSIS Rank Report:")
print(rank_report)

TOPSIS Rank Report:
     Model  Closeness Coefficient  Rank
0     BART               0.839756     1
1  PEGASUS               0.753906     2
4      LED               0.471522     3
2       T5               0.393957     4
3    GPT-3               0.000000     5
